In [ ]:
%load_ext autoreload
%autoreload 2

# Download 17-18.05.21

In [ ]:
from src.data.download import download_whole_city
from src.settings import DATA_RAW_DIR
from tqdm import tqdm

In [ ]:
RESOLUTION = 9

In [ ]:
cities = [
    "Tirana, Albania",
    "Yerevan, Armenia",
    "Vienna, Austria",
    "Baku, Azerbaijan",
    "Minsk, Belarus",
    "Brussels, Belgium",
    "Sarajevo, Bosnia and Herzegovina",
    "Sofia, Bulgaria",
    "Zagreb, Croatia",
    "Nicosia, Cyprus",
    "Prague, Czech Republic",
    "Tallinn, Estonia",
    "Helsinki, Finland",
    "Paris, France",
    "Tbilisi, Georgia",
    "Berlin, Germany",
    "Budapest, Hungary",
    "Reykjavík, Iceland",
    "Dublin, Ireland",
    "Rome, Italy",
    "Nur-Sultan, Kazakhstan",
    "Latvia, Riga",
    "Vilnius, Lithuania",
    "Luxembourg City, Luxembourg",
    "Valletta, Malta",
    "Chișinău, Moldova",
    "Podgorica, Montenegro",
    "Amsterdam, Netherlands",
    "Skopje, North Macedonia",
    "Oslo, Norway",
    "Warszawa, PL",
    "Kraków, PL",
    "Łódź, PL",
    "Wrocław, PL",
    "Poznań, PL",
    "Gdańsk, PL",
    "Szczecin, PL",
    "Lisbon, Portugal",
    "Bucharest, Romania",
    ['Moscow, Russia', 'Zelenogradsky Administrative Okrug', 'Western Administrative Okrug', 'Novomoskovsky Administrative Okrug', 'Troitsky Administrative Okrug'],
    "Belgrade, Serbia",
    "Bratislava, Slovakia",
    "Ljubljana, Slovenia",
    "Madrid, Spain",
    "Stockholm, Sweden",
    "Bern, Switzerland",
    "Ankara, Turkey",
    "Kyiv, Ukraine",
    ["London, United Kingdom", "City of London"],
    "New York City, USA",
    "Chicago, USA",
    "Los Angeles, USA",
    "San Francisco, USA",
    "Philadelphia, USA",
]

In [ ]:
for city in tqdm(cities):
    download_whole_city(city, DATA_RAW_DIR)

# Process dataset, select tags, add h3 indices of selected resolution

## Append h3 indices of selected resolution

In [ ]:
from src.data.make_dataset import add_h3_indices_to_city

In [ ]:
for city in tqdm(cities):
    add_h3_indices_to_city(city, 8)

## Group selected tags in cities

In [ ]:
from src.data.load_data import load_filter
from src.data.make_dataset import group_city_tags

In [ ]:
cities = [
    "Vienna, Austria",
    "Minsk, Belarus",
    "Brussels, Belgium",
    "Sofia, Bulgaria",
    "Zagreb, Croatia",
    "Prague, Czech Republic",
    "Tallinn, Estonia",
    "Helsinki, Finland",
    "Paris, France",
    "Berlin, Germany",
    "Reykjavík, Iceland",
    "Dublin, Ireland",
    "Rome, Italy",
    "Nur-Sultan, Kazakhstan",
    "Latvia, Riga",
    "Vilnius, Lithuania",
    "Luxembourg City, Luxembourg",
    "Amsterdam, Netherlands",
    "Oslo, Norway",
    "Warszawa, PL",
    "Kraków, PL",
    "Łódź, PL",
    "Wrocław, PL",
    "Poznań, PL",
    "Gdańsk, PL",
    "Lisbon, Portugal",
    "Moscow, Russia",
    "Belgrade, Serbia",
    "Bratislava, Slovakia",
    "Ljubljana, Slovenia",
    "Madrid, Spain",
    "Stockholm, Sweden",
    "Bern, Switzerland",
    "London, United Kingdom",
    "New York City, USA",
    "Chicago, USA",
    "San Francisco, USA",
]

In [ ]:
SELECTED_TAGS = [
    "aeroway",
    "amenity",
    "building",
    "healthcare",
    "historic",
    "landuse",
    "leisure",
    "military",
    "natural",
    "office",
    "shop",
    "sport",
    "tourism",
    "water",
    "waterway",
]

In [ ]:
key_values_to_drop = {
    "natural": [
        "tree",
        "tree_row",
        "valley",
        'scrub',
        'heath',
        'moor',
        'grassland',
        'fell',
        'bare_rock',
        'scree',
        'shingle',
        'sand',
        'wetland',
        'glacier',
        'reef',
        'spring',
        'hot_spring',
        'geyser',
        'peak',
        'volcano',
        'peninsula',
        'isthmus',
        'ridge',
        'arete',
        'cliff',
        'saddle',
        'dune',
        'rock',
        'stone',
        'sinkhole',
        'cave_entrance',
    ],
    "amenity": [
        'bench',
        'grit_bin',
        'parking_space',
        'post_box',
        'sanitary_dump_station',
        'recycling',
        'waste_basket',
        'waste_disposal',
        'waste_transfer_station',
    ],
    "landuse": [
        "grass"
    ]
}

In [ ]:
TAG_FILTER = load_filter("from_wiki.json")

In [ ]:
for city in tqdm(cities):
    group_city_tags(city, 8, filter_values=TAG_FILTER, fill_missing=True)

# Group all cities

In [ ]:
from src.data.make_dataset import group_cities

df = group_cities(cities, RESOLUTION)


In [ ]:
df['city'].value_counts()

In [ ]:
from keplergl import KeplerGl
KeplerGl(data={"cities": df})

In [ ]:
city_buildings = df[['city', *[c for c in df.columns if c.startswith("building")]]].groupby(['city']).sum()

In [ ]:
city_buildings.reset_index().set_index('city')

In [ ]:
city_buildings_percent = city_buildings.div(city_buildings.sum(axis=1), axis=0).reset_index()
city_buildings_percent

In [ ]:
melted = city_buildings_percent.melt(id_vars = 'city').sort_values(by=['city', 'value'], ascending=[True, False])
melted

In [ ]:
melted[melted['variable'] == 'building_yes']

In [ ]:
szczecin = df[df['city']=='Szczecin, PL'][['h3', *[c for c in df.columns if c.startswith('building')]]]
KeplerGl(data={'szczecin': szczecin})

In [ ]:
szczecin.drop(columns='h3', ).melt().sort_values('value', ascending=False)